Os dados podem ser baixados em https://www.kaggle.com/datasets/carolzhangdc/imdb-5000-movie-dataset?resource=download 

## Importando as bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

## Leitura dos dados e visualização do conteúdo

In [ ]:
#carrega os dados no dataframe
df = pd.read_csv('movie_metadata.csv')

In [ ]:
#lista as colunas do dataframe
df.columns


In [ ]:
#visualiza as primeiras 5 entradas do dataframe (se quiser ver mais é só colocar um valor dentro dos parênteses)
df.head()

In [ ]:
#imprime o número de linhas e colunas do dataframe
df.shape

In [ ]:
#imprime o tipo de dado em cada coluna: object - variáveis categóricas, float64 e int64 - variáveis numéricas
df.dtypes

In [ ]:
numericas_cols = [col for col in df.columns if df[col].dtype != 'object']
categoricas_cols = [col for col in df.columns if df[col].dtype == 'object']

numericas_cols, categoricas_cols

## Análise Exploratória

In [ ]:
df[numericas_cols].describe()

In [ ]:
df[categoricas_cols].describe()

In [ ]:
#descartando a coluna com o link do IMDB do filme
df.drop('movie_imdb_link', axis=1, inplace=True)

In [ ]:
#Verificando quais os valores da coluna 'color'
df['color'].value_counts()

In [ ]:
#descartando a coluna 'color'

df.drop('color', axis=1, inplace=True)

In [ ]:
#verificando se existem valores faltantes nos dados
df.isna().any()

In [ ]:
#verificando quantos valores faltantes existem nos dados
df.isna().sum()

In [ ]:
#descartando as linhas que tem poucos valores faltantes
df.dropna(axis=0, subset=['director_name', 'num_critic_for_reviews',
                               'duration','director_facebook_likes','actor_3_facebook_likes',
                               'actor_2_name','actor_1_facebook_likes','actor_1_name','actor_3_name',
                               'facenumber_in_poster','num_user_for_reviews','language','country',
                               'actor_2_facebook_likes','plot_keywords', 'title_year'],inplace=True)

In [ ]:
#verificando quantas linhas de dados ainda estão no dataframe. Perdemos apenas 6% dos dados
df.shape

In [ ]:
#verificando quais os valores na coluna 'content rating' (classificação indicativa)
df['content_rating'].value_counts()

In [ ]:
#substituindo os valores faltantes por 'Not Rated'
df['content_rating'].fillna('Not Rated', inplace=True)

In [ ]:
##verificando quais os valores na coluna 'aspect ratio' (proporção de tela)
df['aspect_ratio'].value_counts()

In [ ]:
#substituindo os valores faltantes pela mediana dos valores - cuidado ao 
#substituir pela mediana, você pode criar valores que não fazem sentido pro seu problema
df['aspect_ratio'].fillna(df['aspect_ratio'].median(), inplace=True)

In [ ]:
##verificando quais os valores na coluna 'gross' (ganho bruto do filme)

df['gross'].value_counts()

In [ ]:
#substituindo os valores faltantes pela mediana dos valores
df['gross'].fillna(df['gross'].median(), inplace=True)

In [ ]:
df['budget'].value_counts()

In [ ]:
#substituindo os valores faltantes pela mediana dos valores
df['budget'].fillna(df['budget'].median(), inplace=True)

In [ ]:
#verificando se removemos (ou substituimos) todos os valores faltantes
df.isna().sum()

In [ ]:
#verificando se temos valores duplicados
df.duplicated().sum()

In [ ]:
#removendo as duplicatas
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
#verificando quais os valores da coluna 'language'
df['language'].value_counts()

In [ ]:
#descartando a coluna 'language'
df.drop('language', axis=1, inplace=True)

In [ ]:
#verificando os valores na coluna 'country'
df['country'].value_counts()

In [ ]:
#descartando a coluna 'country'
df.drop('country', axis=1, inplace=True)

In [ ]:
#criando uma nova coluna na tabela
df['Profit'] = df['budget'].sub(df['gross'], axis=0)

In [ ]:
#verificando se a coluna foi criada corretamente
df.head()

In [ ]:
#salvando os dados do dataframe em um arquivo csv
df.to_csv('dados_imdb_analiseexpl.csv', index=False)

## Visualização dos dados

In [ ]:
#criando gráfico de correlaciona lucro e nota do IMDB
ggplot(aes(x='imdb_score', y='Profit'), data=df) +\
    geom_line() +\
    stat_smooth(colour='blue', span=1)

In [ ]:
#criando gráfico de correlaciona likes no facebook do filme e nota do IMDB

(ggplot(df)+\
    aes(x='imdb_score', y='movie_facebook_likes') +\
    geom_line() +\
    labs(title='Nota no IMDB vs likes no facebook do filme', x='Nota no IMDB', y='Likes no facebook')
)

In [ ]:
#gráfico dos 20 filmes com melhor nota com relação aos atores principais
plt.figure(figsize=(10,8))

df= df.sort_values(by ='imdb_score' , ascending=False)
df2=df.head(20)
ax=sns.pointplot(df2['actor_1_name'], df2['imdb_score'], hue=df2['movie_title'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
n, bins, patches = plt.hist(df['duration'], bins=50)
plt.xlabel('Duração do Filme')
plt.ylabel('Número de Filmes')
plt.show()

In [ ]:
n, bins, patches = plt.hist(df['imdb_score'], bins=50)
plt.xlabel('Nota do Filme')
plt.ylabel('Número de Filmes')
plt.show()


In [ ]:
#outra forma de verificar a distribuição dos dados de score
plt.boxplot(df['imdb_score'])

## Preparação dos dados

In [ ]:
#retirando algumas colunas com dados categóricos
df.drop(columns=['director_name', 'actor_1_name', 'actor_2_name', 
                 'actor_3_name', 'plot_keywords', 'movie_title'], axis=1, inplace=True)

In [ ]:
#verificando os valores da coluna 'genre'
df['genres'].value_counts()

In [ ]:
#retirando a coluna 'genres'
df.drop('genres', axis=1, inplace=True)

In [ ]:
#retirando a coluna criada
df.drop(columns=['Profit'], axis=1, inplace=True)

In [ ]:
#verificando se existem colunas fortemente correlacionadas
import numpy as np
corr = df.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.0})
plt.figure(figsize=(20,10))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, 1)] = True
a = sns.heatmap(corr,mask=mask, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
#criando uma nova coluna combinando as duas colunas muito correlacionadas
df['Other_actors_facebook_likes'] = df['actor_2_facebook_likes'] + df['actor_3_facebook_likes']

In [ ]:
#removendo as colunas
df.drop(columns=['actor_2_facebook_likes', 'actor_3_facebook_likes',
                 'cast_total_facebook_likes'], axis=1, inplace=True)

In [ ]:
#criando uma nova coluna combinando as duas colunas muito correlacionadas

df['critic_review_ratio'] = df['num_critic_for_reviews']/df['num_user_for_reviews']

In [ ]:
#removendo as colunas
df.drop(columns=['num_critic_for_reviews', 'num_user_for_reviews'], axis=1, inplace=True)

In [ ]:
#verificando se ainda existem colunas fortemente correlacionadas

corr = df.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(13,7))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, 1)] = True
a = sns.heatmap(corr,mask=mask, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

In [ ]:
#categorizando os valores de nota do imdb
df['imdb_binned_score']=pd.cut(df['imdb_score'], bins=[0,4,6,8,10], right=True, labels=False)+1

In [ ]:
df['imdb_score'].value_counts()

In [ ]:
df['imdb_binned_score'].value_counts()

In [ ]:
df.head()

In [ ]:
#criando novas colunas para transformar os valores categóricos de 'content rating' (classificação indicativa)
#em valores numéricos
df = pd.get_dummies(data = df, columns=['content_rating'], prefix=['content_rating'], drop_first=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('dados_imdb_com_nota.csv', index=False)

In [ ]:
#escolhendo as colunas do dataframe que serão nossos valores de entrada para o modelo
X=pd.DataFrame(columns=['duration','director_facebook_likes','actor_1_facebook_likes','gross',
                        'num_voted_users','facenumber_in_poster','budget','title_year','aspect_ratio',
                        'movie_facebook_likes','Other_actors_facebook_likes','critic_review_ratio',
                        'content_rating_G','content_rating_GP',
                        'content_rating_M','content_rating_NC-17','content_rating_Not Rated',
                        'content_rating_PG','content_rating_PG-13','content_rating_Passed',
                        'content_rating_R','content_rating_TV-14','content_rating_TV-G',
                        'content_rating_TV-PG','content_rating_Unrated','content_rating_X'],data=df)

In [ ]:
#escolhendo a(s) coluna(s) do dataframe que serão a resposta do modelo
y = pd.DataFrame(columns=['imdb_binned_score'], data=df)